In [1]:
# from https://github.com/lh3/readfq/blob/master/readfq.py
def readfq(fp): # this is a generator function
    last = None # this is a buffer keeping the last unprocessed line
    while True: # mimic closure; is it a bad idea?
        if not last: # the first record or a record following a fastq
            for l in fp: # search for the start of the next record
                if l[0] in '>@': # fasta/q header line
                    last = l[:-1] # save this line
                    break
        if not last: break
        name, seqs, last = last[1:].partition(" ")[0], [], None
        for l in fp: # read the sequence
            if l[0] in '@+>':
                last = l[:-1]
                break
            seqs.append(l[:-1])
        if not last or last[0] != '+': # this is a fasta record
            yield name, ''.join(seqs), None # yield a fasta record
            if not last: break
        else: # this is a fastq record
            seq, leng, seqs = ''.join(seqs), 0, []
            for l in fp: # read the quality
                seqs.append(l[:-1])
                leng += len(l) - 1
                if leng >= len(seq): # have read enough quality
                    last = None
                    yield name, seq, ''.join(seqs); # yield a fastq record
                    break
            if last: # reach EOF before reading enough quality
                yield name, seq, None # yield a fasta record instead
                break

In [26]:
def rev_cmp(s):
    d = {'A':'T', 'C':'G', 'G':'C', 'T':'A', 'N': 'N'}
    return ''.join( [d[c] for c in reversed(s)] )

In [2]:
index = {}
k = 13
for name, seq, _ in readfq(open('sars_cov2.fa', 'r')):
    for kpos in range(0, len(seq)-k):
        km = seq[kpos:kpos+k]
        if km in index:
            index[km].append(kpos)
        else:
            index[km] = [kpos]

In [3]:
index

{'ATTAAAGGTTTAT': [0],
 'TTAAAGGTTTATA': [1],
 'TAAAGGTTTATAC': [2],
 'AAAGGTTTATACC': [3],
 'AAGGTTTATACCT': [4],
 'AGGTTTATACCTT': [5],
 'GGTTTATACCTTC': [6],
 'GTTTATACCTTCC': [7],
 'TTTATACCTTCCC': [8],
 'TTATACCTTCCCA': [9],
 'TATACCTTCCCAG': [10],
 'ATACCTTCCCAGG': [11],
 'TACCTTCCCAGGT': [12],
 'ACCTTCCCAGGTA': [13],
 'CCTTCCCAGGTAA': [14],
 'CTTCCCAGGTAAC': [15],
 'TTCCCAGGTAACA': [16],
 'TCCCAGGTAACAA': [17],
 'CCCAGGTAACAAA': [18],
 'CCAGGTAACAAAC': [19],
 'CAGGTAACAAACC': [20],
 'AGGTAACAAACCA': [21],
 'GGTAACAAACCAA': [22],
 'GTAACAAACCAAC': [23],
 'TAACAAACCAACC': [24],
 'AACAAACCAACCA': [25],
 'ACAAACCAACCAA': [26],
 'CAAACCAACCAAC': [27],
 'AAACCAACCAACT': [28],
 'AACCAACCAACTT': [29],
 'ACCAACCAACTTT': [30],
 'CCAACCAACTTTC': [31],
 'CAACCAACTTTCG': [32],
 'AACCAACTTTCGA': [33],
 'ACCAACTTTCGAT': [34],
 'CCAACTTTCGATC': [35],
 'CAACTTTCGATCT': [36],
 'AACTTTCGATCTC': [37],
 'ACTTTCGATCTCT': [38],
 'CTTTCGATCTCTT': [39],
 'TTTCGATCTCTTG': [40],
 'TTCGATCTCTTGT': [41],
 '

In [35]:
reads = []
for name, seq, qual in readfq(open('test_reads.fq', 'r')):
    reads.append((name, seq))

In [36]:
def lookup_seeds(read, index, k=13):
    hits = []
    rseq = read[1]
    for p in range(0, len(rseq)-k):
        km = rseq[p:p+k]
        if km in index:
            for hit in index[km]:
                hits.append((p, hit))
    return hits

In [42]:
res = lookup_seeds(reads[0], index)

In [43]:
def res_summary(res):
    map_pos = set([])
    for q,r in res:
        map_pos.add(r-q)
    return map_pos

In [45]:
print(res_summary(res))

{25, 26, 27}


### Minimizers

In [56]:
import xxhash

In [130]:
def rand_hash(x): return xxhash.xxh3_64_digest(x, seed=2212024)

In [131]:
def get_minimizers(s, w, k, hashfn = lambda x: x):
    l = w + k - 1
    minimizers = []
    for window_start in range(0, len(s) - l):
        window = s[window_start:window_start+l]
        minimizer = sorted([(hashfn(window[p:p+k]), p+window_start, window[p:p+k]) for p in range(0, len(window)-k)])[0]
        if len(minimizers) == 0:
            minimizers.append(minimizer[1:])
        elif minimizer[1:] != minimizers[-1]:
            minimizers.append(minimizer[1:])
    return minimizers

In [132]:
min_index = {}
for name, seq, _ in readfq(open('sars_cov2.fa', 'r')):
    for p,m in get_minimizers(seq, 20, 13):
        if m in min_index:
            min_index[m].append(p)
        else:
            min_index[m] = [p]

In [133]:
min_index_rand = {}
for name, seq, _ in readfq(open('sars_cov2.fa', 'r')):
    for p,m in get_minimizers(seq, 20, 13, rand_hash):
        if m in min_index_rand:
            min_index_rand[m].append(p)
        else:
            min_index_rand[m] = [p]

Since we are indexing only the minimizer positions, the size of the index reduces substantially!

In [134]:
len(index)

29835

In [135]:
len(min_index)

3445

In [136]:
len(min_index_rand)

3001

In [137]:
def total_inv_list_size(d):
    return sum([len(v) for k,v in d.items()])

In [138]:
total_inv_list_size(index)

29890

In [139]:
total_inv_list_size(min_index)

3450

In [140]:
total_inv_list_size(min_index_rand)

3005